In [27]:
#Import libraries/Dependencies
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from IPython.display import clear_output
%matplotlib inline

In [28]:
np.random.seed(42)

# Portfolio Planner

In this activity, you will use the Alpaca api to grab historical data for a 60/40 portfolio using `SPY` to represent the stock portion and `AGG` to represent the bonds.

In [29]:
# Load .env enviroment variables
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY3")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY3")

# api = tradeapi.REST()
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

ValueError: ('Key ID must be given to access Alpaca trade API', ' (env: APCA_API_KEY_ID)')

In [30]:
#get all assets
assets = api.list_assets()
assets[0]

#kep tradeable  assets
tradeable = [asset for asset in assets if asset.tradable ]

NameError: name 'api' is not defined

# Data Collection

In this step, you will need to use the Alpaca api to fetch closing prices for the `SPY` and `AGG` tickers. Save the results as a pandas DataFrame

In [31]:
asset_info_df = pd.DataFrame()
asset_info_df['symbol'] = pd.Series([asset.symbol for asset in assets])
display(asset_info_df.head(10))

NameError: name 'assets' is not defined

In [32]:
list_of_tickers = ["SPY", "AGG"]
# sert time frame
timeframe  = '1D'
#set the start and end datetimes for year, now and 365 days ago
end_date = datetime.now()
start_date = end_date + timedelta(-365)

# get 1 year of historical data from api
df = api.get_barset(
    list_of_tickers,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,).df

#drop extraneous columns and outer tables
#df = df.droplevel(axis=1, level=0)

#use the drop function to drop extra cloumns
df.drop(columns=['open', 'high', 'low', 'volume'], level = 1, inplace=True)

#daily data. keep only date, and remove time, component of data
df.index = df.index.date
df.head()

NameError: name 'api' is not defined

In [33]:
# Calculate the daily roi for the stocks
daily_returns = df.pct_change()
daily_returns

NameError: name 'df' is not defined

In [34]:
daily_returns.dropna()

NameError: name 'daily_returns' is not defined

In [35]:
# Calculate volatility
spy_daily_vol = daily_returns.std()['SPY']['close']
agg_daily_vol= daily_returns.std()['AGG']['close']
print(agg_daily_vol)
print(spy_daily_vol)

NameError: name 'daily_returns' is not defined

In [36]:
# Save the last day's closing price
spy_last_days_price = df['SPY']['close'][-1]
agg_last_days_price = df['AGG']['close'][-1]
print(spy_last_days_price)
print(agg_last_days_price)

NameError: name 'df' is not defined

In [37]:
# Setup the Monte Carlo Parameters
number_simulations = 500
num_records = 252 * 30
monte_carlo = pd.DataFrame()
portfolio_cumulative_returns = pd.DataFrame()

In [38]:
for x in range(number_simulations):
    
    print(f"Simulation {x}")
    
    spy_prices=[spy_last_days_price]
    agg_prices=[agg_last_days_price]
    
    for i in range(num_records):
        #was $100 -> $103.82 -> $100
        new_spy_price = spy_prices[-1] * (1 + np.random.normal(daily_returns.mean()['SPY']['close'],spy_daily_vol))
        new_agg_price = agg_prices[-1] * (1 + np.random.normal(daily_returns.mean()['AGG']['close'],agg_daily_vol))
        
        spy_prices.append(new_spy_price)
        agg_prices.append(new_agg_price)
    
    #list of price changes
    cumulative = pd.DataFrame({"SPY":spy_prices,"AGG":agg_prices})
    
    #stock returns
    returns = cumulative.pct_change()
    
    weight1 = 0.6
    weight2 = 0.4
    
    portfolio_returns = weight1 * returns['SPY'] + weight2 * returns['AGG']
    
    portfolio_cumulative_returns[x] = (1 + portfolio_returns.fillna(0)).cumprod()
    
    #clear out the list for you so you dont have to scroll down to 500
    
    clear_output(wait=True)
    
portfolio_cumulative_returns.head()
          
    
    
    


Simulation 0


NameError: name 'spy_last_days_price' is not defined

In [ ]:
plot_title = f"{number_simulations} Simulations"
portfolio_cumulative_returns.plot(legend = None, title = plot_title, figsize = (20, 10))
# Visualize the Simulation
# YOUR CODE HERE

In [18]:
ending_portfolio_cumulative_returns = portfolio_cumulative_returns.iloc[-1,:]
                                                                       
ending_portfolio_cumulative_returns
# Select the last row for the cumulative returns (cumulative returns at 30 years)
# YOUR CODE HERE

IndexError: single positional indexer is out-of-bounds

In [39]:
# Select the last row for the cumulative returns (cumulative returns at 20 years)
ending_portfolio_cumulative_returns = portfolio_cumulative_returns.iloc[20*252,:]
ending_portfolio_cumulative_returns

IndexError: single positional indexer is out-of-bounds

In [40]:
# Display the 90% confidence interval for the ending returns
confidence_interval_30_years =  ending_portfolio_cumulative_returns.quantile(q=[.05, .95, .5])
confidence_interval_30_years

NameError: name 'ending_portfolio_cumulative_returns' is not defined

In [41]:
# Visualize the distribution of the ending returns - (K = 1 + 3. 322 logN)
#plot_title = f"{number_simulations} Simulations"
import math

bins = round(1+(3.222 * math.log(number_simulations)))
ending_portfolio_cumulative_returns.plot(kind = 'hist', title = plot_title, figsize = (20, 10), bins = 50)
plt.axvline(confidence_interval_30_years.iloc[0], color = 'r')
plt.axvline(confidence_interval_30_years.iloc[1], color = 'r')

NameError: name 'ending_portfolio_cumulative_returns' is not defined

In [42]:
round(1+(3.222 * math.log(500)))

21

---

# Retirement Analysis

In this section, you will use the monte carlo model to answer the following retirement planning questions:

1. What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?
2. Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?
3. Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?
4. How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

### What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?

In [23]:
expected_return = ending_portfolio_cumulative_returns.quantile(q=[.1, .5, .95])
expected_return

NameError: name 'ending_portfolio_cumulative_returns' is not defined

### Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?

In [43]:
# YOUR CODE HERE
investment = 20000
#int(confidence_interval_30_years.iloc[0]* investment)
thirty_year_return = expected_return * investment
thirty_year_return

NameError: name 'expected_return' is not defined

### Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?

Note: This is effectively saying that 90% of the expected returns will be greater than the return at the 10th percentile, so this can help measure the uncertainty about having enough funds at retirement

In [44]:
# YOUR CODE HERE
income = 7285

#withdraw = .4 * (portfolio at the end of 30 years, 10th percentile) = .4 * (thirty_year_return[0]) = .4 * (.1 quartile * investment)
withdraw = .04 * (ending_portfolio_cumulative_returns.quantile(q=[.1]) * investment).iloc[0]

if withdraw >= income:
    print("You can retire")
else:
    print("Keep working")

NameError: name 'ending_portfolio_cumulative_returns' is not defined

### How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

In [45]:
# # YOUR CODE HERE
income = 7285

# AB + AC = A * (B + C)
# 2*3 + 2*4 = 2 * (3 + 4)
#withdraw = .4 * (portfolio at the end of 30 years, 10th percentile) = .4 * (thirty_year_return[0]) = .4 * (.1 quartile * investment)
#100 + tax = 100 + 100 * (8.00%) = 100 * 1 + 100 * (0.08) = 100 * (1 + .08) = 100 * (1.08)

withdraw = .04 * (ending_portfolio_cumulative_returns.quantile(q=[.1]) * (1.50) * investment).iloc[0]

if withdraw >= income:
    print(f"You can retire because your retirement income will be {withdraw}")
else:
    print(f"Keep working because your retirement income  is {withdraw}")

NameError: name 'ending_portfolio_cumulative_returns' is not defined

### Optional Challenge

In this section, you will calculate and plot the cumulative returns for the median and 90% confidence intervals. This plot shows the expected cumulative returns for any given day between the first day and the last day of investment. 

# YOUR CODE HERE